# Import packages

In [1]:
import networkx as nx
import cobra 
import os
import glob
import pandas as pd
import sys
sys.path.append('../code/')
from path_analysis_function import *

# Input and output files

In [2]:
pairs_of_currency_metabolites_file = "../data/external/pairs_of_currency_metabolites.csv" #the file stores the currency metabolite pairs
special_currency_metabolites_file = "../data/external/special_currency_metabolites.csv" #the file stores the special currency metabolites
modelfile = '../data/external/model/'   #folder containing the models
interim_save_path = '../data/interim/bowtie_graph/' #folder containing intermediate results
result_save_path = '../data/result/bowtie_graph/' # Bowtie result folders for different models
biggid2name=pd.read_csv("../data/external/biggid2name.csv",index_col=0)#All metabolite ID and Name of BIGG
bowtie_graph_outputfile = result_save_path+'bowtie_graph_total_output.csv' # Bowtie result for different models
bowtie_graph_analysis_tablefile = '../data/result/bowtie_graph/bowtie_graph_analysis_table.csv' #statistics table

# Get currency metabolite from csv file

In [3]:
pairs_of_currency_metabolites = pd.read_csv(pairs_of_currency_metabolites_file)
special_currency_metabolites = pd.read_csv(special_currency_metabolites_file)

pi_pairs1 = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['pi_pairs1'] if str(x) != 'nan']
pi_pairs2 = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['pi_pairs2'] if str(x) != 'nan']
h_pairs1 = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['h_pairs1'] if str(x) != 'nan']
h_pairs2 = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['h_pairs2'] if str(x) != 'nan']
nh4_pairs = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['nh4_pairs'] if str(x) != 'nan']
other_pairs = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['other_pairs'] if str(x) != 'nan']

currency_mets = [x for x in special_currency_metabolites['excluded'] if str(x) != 'nan']

# Get bow tie structure

In [4]:
modelfile_list = sorted(glob.glob(modelfile+'*.xml'), reverse=False)
for use_model_file in modelfile_list:
    model_name = use_model_file.split('/')[-1].split('.')[0]
    if not os.path.exists(interim_save_path):
        os.makedirs(interim_save_path)
    #covert the model to a graph
    resultfile=interim_save_path+ model_name+'_metabolite_links.txt'
    model = cobra.io.read_sbml_model(use_model_file)
    G=nx.MultiDiGraph()
    for rea in model.reactions:
        if not rea.boundary and 'BIOMASS' not in rea.id: 
            sub_pro = get_metpair(rea,pi_pairs1,h_pairs1,pi_pairs2,h_pairs2,nh4_pairs,other_pairs,currency_mets) 
            for sp in sub_pro:
                G.add_edge(sp[0], sp[1], label=rea.id)
                if rea.reversibility:  
                    G.add_edge(sp[1], sp[0], label=rea.id)
    nx.write_edgelist(G, resultfile) 
    
    #calculate bowtie 
    if not os.path.exists(result_save_path):
        os.makedirs(result_save_path)
    btfile=result_save_path+ model_name+'_bowtie.txt'
    bowtiefile=open(btfile,'w')
    gsc=list(max(nx.strongly_connected_components(G), key=len)) #giant strong component,convert returned set to list
    n=gsc[0] #select a node in GSC
    outdm=set(nx.single_source_shortest_path_length(G,n).keys()) #output domain     #Compute the shortest path lengths from source to all reachable nodes.
    indm=set(nx.single_source_shortest_path_length(G.reverse(),n).keys()) #input domain
    outs=list(outdm-indm) #use sets to find intersection, union and difference between two lists
    ins=list(indm-outdm)
    allnodes=set(G.nodes())
    isset=list(allnodes-(indm|outdm))
    for node in gsc:
        bowtiefile.write(node+'\tGSC\n')
    for node in ins:
        bowtiefile.write(node+'\tIN\n')
    for node in outs:       
        bowtiefile.write(node+'\tOUT\n')
    for node in isset:
        bowtiefile.write(node+'\tIS\n')
    bowtiefile.close() 
    print(model_name,str(len(gsc)),str(len(outs)),str(len(ins)))
print('finish')

iAF692 247 195 32
iAF987 548 242 88
iAM_Pb448 508 109 79
iAM_Pc455 514 105 81
iAM_Pf480 516 105 82
iAM_Pk459 514 107 81
iHN637 294 177 59
iJN1463 933 223 594
iML1515 906 230 411
iPC815 615 212 233
iSDY_1059 782 276 247
iSF_1195 771 271 364
iSSON_1240 810 265 383
iSbBS512_1146 802 254 288
iYL1228 763 201 379
finish


In [5]:
#Read the Bowtie table, extract each column of metabolites, and determine the category of metabolites in the model
#Organize the resulting file into a Bowtie
globfiles = sorted(glob.glob(result_save_path+'*_bowtie.txt'), reverse=False) 
#print(globfile1)
for filename in globfiles:
    print(filename)
    #model_name = use_model_file.split('\\')[1].split('.')[0] #for windows
    model_name = filename.split('/')[-1].split('_bowtie')[0]
    model_bowtie = pd.read_csv(filename,names=['bowtie'],sep='\t')
    for index, row in model_bowtie.iterrows():   
        if index in biggid2name.index:
            biggid2name.loc[index,model_name]=row['bowtie']

biggid2name.to_csv(bowtie_graph_outputfile, header=True, index=True,mode='w')
print('finish')


../data/result/bowtie_graph/iAF692_bowtie.txt
../data/result/bowtie_graph/iAF987_bowtie.txt
../data/result/bowtie_graph/iAM_Pb448_bowtie.txt
../data/result/bowtie_graph/iAM_Pc455_bowtie.txt
../data/result/bowtie_graph/iAM_Pf480_bowtie.txt
../data/result/bowtie_graph/iAM_Pk459_bowtie.txt
../data/result/bowtie_graph/iHN637_bowtie.txt
../data/result/bowtie_graph/iJN1463_bowtie.txt
../data/result/bowtie_graph/iML1515_bowtie.txt
../data/result/bowtie_graph/iPC815_bowtie.txt
../data/result/bowtie_graph/iSDY_1059_bowtie.txt
../data/result/bowtie_graph/iSF_1195_bowtie.txt
../data/result/bowtie_graph/iSSON_1240_bowtie.txt
../data/result/bowtie_graph/iSbBS512_1146_bowtie.txt
../data/result/bowtie_graph/iYL1228_bowtie.txt
finish


In [6]:
modelfile_list = sorted(glob.glob(modelfile+'*.xml'), reverse=False)
bowtie_graph_data=pd.read_csv(bowtie_graph_outputfile,index_col=0)
bowtie_analysis_table = pd.DataFrame()
for eachmodelfile in modelfile_list:
    #model_name = eachmodelfile.split('\\')[1].split('.')[0] #for windows
    model_name = eachmodelfile.split('/')[-1].split('.')[0]
    print(model_name)
    record_has_data = [x for x in bowtie_graph_data[model_name] if str(x) != 'nan']
    bowtie_analysis_table.loc[model_name,'All Metabolites']=len(record_has_data)
    bowtie_analysis_table.loc[model_name,'GSC']=record_has_data.count('GSC')
    bowtie_analysis_table.loc[model_name,'IN']=record_has_data.count('IN')
    bowtie_analysis_table.loc[model_name,'OUT']=record_has_data.count('OUT')
    bowtie_analysis_table.loc[model_name,'IS']=record_has_data.count('IS')

bowtie_analysis_table.to_csv(bowtie_graph_analysis_tablefile, header=True, index=True,mode='w')
print('finish')

iAF692
iAF987
iAM_Pb448
iAM_Pc455
iAM_Pf480
iAM_Pk459
iHN637
iJN1463
iML1515
iPC815
iSDY_1059
iSF_1195
iSSON_1240
iSbBS512_1146
iYL1228
finish
